In [27]:
import os
from dotenv import load_dotenv
load_dotenv('key.env')  

key_string = os.getenv('open_ai_API_Key')

In [28]:
import re
from langchain.document_loaders import PyPDFLoader
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate

In [29]:
# Load PDF
loader_1 = PyPDFLoader("apple-2024.pdf")
document_1 = loader_1.load()
loader_2 = PyPDFLoader("apple-2022.pdf")
document_2 = loader_2.load()
loader_3 = PyPDFLoader("apple-2020.pdf")
document_3 = loader_3.load()
loader_4 = PyPDFLoader("apple-2018.pdf")
document_4 = loader_4.load()
loader_5 = PyPDFLoader("apple-2016.pdf")
document_5 = loader_5.load()

In [56]:
def page_finder(document):
    # Initialize variables to store page numbers
    start_page = None
    end_page = None

    # Find the page number where content starts with "Item 8"
    for doc in document:
        if doc.page_content.startswith("Item 8"):
            start_page = doc.metadata['page']
            break  # Exit the loop after finding the first match

    # Find the page number where content starts with "Item 9"
    for doc in document:
        if doc.page_content.startswith("Item 9"):
            end_page = doc.metadata['page']
            break  # Exit the loop after finding the first match

    # If either page is still None (should not happen if items are guaranteed)
    if start_page is None or end_page is None:
        raise ValueError("Could not find 'Item 8' or 'Item 9' in the document")

    return start_page, end_page


def page_executor(document):
    # Find the start and end pages
    start_page, end_page = page_finder(document)

    # Extract documents between the start and end pages
    extracted_documents = [
        doc for doc in document
        if start_page <= doc.metadata['page'] < end_page
    ]

    return extracted_documents

In [43]:
extract_2024 = page_executor(document_1)

In [44]:
extract_2024

[Document(metadata={'source': 'apple-2024.pdf', 'page': 30}, page_content='Item 8.    Financial Statements and Supplementary Data\nIndex to Consolidated Financial Statements Page\nConsolidated Statements of Operations for the years ended September 28, 2024, September 30, 2023 and September 24, 2022 29\nConsolidated Statements of Comprehensive Income for the years ended September 28, 2024, September 30, 2023 and September 24, 2022 30\nConsolidated Balance Sheets as of September 28, 2024 and September 30, 2023 31\nConsolidated Statements of Shareholders’  Equity for the years ended September 28, 2024, September 30, 2023 and September 24, 2022 32\nConsolidated Statements of Cash Flows for the years ended September 28, 2024, September 30, 2023 and September 24, 2022 33\nNotes to Consolidated Financial Statements 34\nReports of Independent Registered Public Accounting Firm 48\nAll financial statement schedules have been omitted, since the required information is not applica ble or is not pr

In [45]:
extract_2022 = page_executor(document_2)
extract_2020 = page_executor(document_3)
extract_2018 = page_executor(document_4)
extract_2016 = page_executor(document_5)

ValueError: Could not find 'Item 8' or 'Item 9' in the document

In [60]:
def page_executor(document):
    # Find the start and end pages
    start_page = 40
    end_page = 46

    # Extract documents between the start and end pages
    extracted_documents = [
        doc for doc in document
        if start_page <= doc.metadata['page'] < end_page
    ]

    return extracted_documents

In [61]:
extract_2016 = page_executor(document_5)

In [62]:
extract_2016

[Document(metadata={'source': 'apple-2016.pdf', 'page': 40}, page_content='Item 8.Financial Statement s and Supplementary DataIndex to Consolidated Financial Statements\n\xa0 Page Consolidated Statements of Operation\ns for the years ended September 24, 2016, September 26, 2015 and   September 27, 2014\xa0 39 Consolidated Statements of Comp\nrehensive Income for the years ended September 24, 2016, September 26, 2015   and September 27, 2014\xa0 40 Consolidated Balance Sheets a\ns of September 24, 2016 and September 26, 2015\xa0 41 Consolidated Statements of Shareho\nlders’ Equity for the years ended September 24, 2016, September 26, 2015 and September 27, 2014\xa0 42 Consolidated Statements of Cash F\nlows for the years ended September 24, 2016, September 26, 2015 and   September 27, 2014\xa0 43 Notes to Consolidated Fin\nancial Statements\xa0 44 Selected Quarterly Financial \nInformation (Unaudited)\xa0 69 Reports of Ernst & Young LLP, Independent Reg\nistered Public Accounting Firm\x

In [ ]:
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, Tool
from langchain.agents import ZeroShotAgent

# Define the system prompt
extract_prompt = PromptTemplate(
    input_variables=["document"],
    template="""
    The following is a section of a 10-K filing:
    {document}

    Extract the STATEMENTS OF OPERATIONS as a structured table. Include column names and all rows. Combine the Table for 10years of data
    Provide the table as a JSON object.

    strucutre the table as follows:
    Item, Category, Subcategory, <Date-1>,<Date-2>,<Date-3>...
    """
)

llm = ChatOpenAI(model="gpt-4o",temperature=0.5, openai_api_key=key_string)

# Define the first chain
extract_chain = LLMChain(
    llm=llm,
    prompt=extract_prompt,
    output_key="balance_sheet",  # This key will pass to the next chain
)

# Define the second prompt
convert_prompt = PromptTemplate(
    input_variables=["balance_sheet"],
    template="""
    Convert the following STATEMENTS OF OPERATIONS  JSON into a TSV format. Return only the TSV data. Separate the 4 tables with headers

    JSON:
    {balance_sheet}
    """
)

# Define the second chain
convert_chain = LLMChain(
    llm=llm,
    prompt=convert_prompt,
    output_key="csv_output",
)

# Combine the chains
sequential_chain = SequentialChain(
    chains=[extract_chain, convert_chain],
    input_variables=["document"],
    output_variables=["csv_output"],
)



In [49]:
a = sequential_chain.run(document = extract_2024)

In [50]:
b = sequential_chain.run(document = extract_2022)

In [52]:
c = sequential_chain.run(document = extract_2020)

In [53]:
d = sequential_chain.run(document = extract_2018)

In [66]:
e = sequential_chain.run(document = extract_2016)

In [67]:
print(e)

```
Category	Subcategory	September 24, 2016	September 26, 2015	September 27, 2014
Net sales		215639	233715	182795
Cost of sales		131376	140089	112258
Gross margin		84263	93626	70537
Operating expenses	Research and development	10045	8067	6041
Operating expenses	Selling, general and administrative	14194	14329	11993
Total operating expenses		24239	22396	18034
Operating income		60024	71230	52503
Other income/(expense), net		1348	1285	980
Income before provision for income taxes		61372	72515	53483
Provision for income taxes		15685	19121	13973
Net income		45687	53394	39510
Earnings per share	Basic	8.35	9.28	6.49
Earnings per share	Diluted	8.31	9.22	6.45
Shares used in computing earnings per share	Basic	5470820	5753421	6085572
Shares used in computing earnings per share	Diluted	5500281	5793069	6122663
Cash dividends declared per share		2.18	1.98	1.82
```


In [71]:
extract_prompt = PromptTemplate(
    input_variables=["document_1","document_2"],
    template="""
    The following are 2 statement of operations.
    {document_1}
    {document_2}


    Combine the 2 tables to 1 table. Make sure to include all the column names and row names in the new table. Return the new statement of operations as a Structured JSON object. Name the object as APPLE. Make sure you get all the data.
    strucutre the table as follows:
    Item, Category, Subcategory, <Date-1>,<Date-2>,<Date-3>...
    """
)

llm = ChatOpenAI(model="gpt-4o",temperature=0.5, openai_api_key=key_string)

# Define the first chain
combine_chain = LLMChain(
    llm=llm,
    prompt=extract_prompt,
    output_key="json",  # This key will pass to the next chain
)

In [72]:
result = combine_chain.run(document_1 = a,document_2 = b )

In [73]:
result_2 = combine_chain.run(document_1 = result,document_2=c)

In [75]:
result_3 = combine_chain.run(document_1 = result_2,document_2=d)

In [76]:
result_4 = combine_chain.run(document_1 = result_3,document_2=e)

In [77]:
print(result_4)

To combine the two tables into a single structured JSON object named "APPLE," we need to ensure all the data from both tables are included, maintaining the structure with columns and rows properly labeled. Here's the combined JSON representation:

```json
{
  "APPLE": [
    {
      "Item": "Net sales",
      "Category": "",
      "Subcategory": "",
      "2024-09-28": 391035,
      "2023-09-30": 383285,
      "2022-09-24": 394328,
      "2021-09-25": 365817,
      "2020-09-26": 274515,
      "2019-09-28": 260174,
      "2018-09-29": 265595,
      "2017-09-30": 229234,
      "2016-09-24": 215639,
      "2015-09-26": 233715,
      "2014-09-27": 182795
    },
    {
      "Item": "Cost of sales",
      "Category": "",
      "Subcategory": "",
      "2024-09-28": 210352,
      "2023-09-30": 214137,
      "2022-09-24": 223546,
      "2021-09-25": 212981,
      "2020-09-26": 169559,
      "2019-09-28": 161782,
      "2018-09-29": 163756,
      "2017-09-30": 141048,
      "2016-09-24": 131376,